In [3]:
import pandas as pd
import h5py
import os
from PIL import Image
import numpy as np

# set path
csv_file_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\train_labels.csv'  # Replace 'your_labels_file.csv' with the name of your CSV file
image_folder = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\train'
hdf5_file_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\matched_images.h5'

# load csv
labels_df = pd.read_csv(csv_file_path)

# create HDF5
with h5py.File(hdf5_file_path, 'w') as hf:
    # images and labels
    num_images = len(labels_df)
    sample_image_path = os.path.join(image_folder, labels_df.iloc[0]['id'] + '.tif')  # assuming the extension is .tif
    image_shape = np.array(Image.open(sample_image_path)).shape
    images_dset = hf.create_dataset("images", shape=(num_images, *image_shape), dtype=np.uint8)
    labels_dset = hf.create_dataset("labels", shape=(num_images,), dtype=np.uint8)
    
    for idx, row in labels_df.iterrows():
        image_path = os.path.join(image_folder, row['id'] + '.tif')  # assuming the extension is .tif
        image_data = np.array(Image.open(image_path))
        
        images_dset[idx] = image_data
        labels_dset[idx] = row['label']


In [4]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

data_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\matched_images.h5'

with h5py.File(data_path, 'r') as hf:
    images = hf['images'][:]
    labels = hf['labels'][:]

# Use stratified sampling to divide the dataset on a 90-5-5 scal
train_images, temp_images, train_labels, temp_labels = train_test_split(
    images, labels, test_size=0.1, random_state=42, stratify=labels
)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

output_hdf5_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\split_data.h5'

with h5py.File(output_hdf5_path, 'w') as hf:

    hf.create_dataset("train_images", data=train_images, dtype=np.uint8)
    hf.create_dataset("train_labels", data=train_labels, dtype=np.uint8)

    hf.create_dataset("val_images", data=val_images, dtype=np.uint8)
    hf.create_dataset("val_labels", data=val_labels, dtype=np.uint8)

    hf.create_dataset("test_images", data=test_images, dtype=np.uint8)
    hf.create_dataset("test_labels", data=test_labels, dtype=np.uint8)


In [3]:
import h5py
import numpy as np

# 函数：裁剪图像的中心32x32区域
def crop_center(img):
    y, x, _ = img.shape
    start_x = x//2 - 16
    start_y = y//2 - 16
    return img[start_y:start_y+32, start_x:start_x+32]

# 从split_data.h5加载数据
with h5py.File('split_data.h5', 'r') as hf:
    train_images = hf['train_images'][:]
    val_images = hf['val_images'][:]
    test_images = hf['test_images'][:]
    train_labels = hf['train_labels'][:]
    val_labels = hf['val_labels'][:]
    test_labels = hf['test_labels'][:]

# 裁剪图像
cropped_train_images = np.array([crop_center(img) for img in train_images])
cropped_val_images = np.array([crop_center(img) for img in val_images])
cropped_test_images = np.array([crop_center(img) for img in test_images])

# 保存裁剪后的数据到新的 HDF5 文件
cropped_hdf5_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\cropped_data.h5'
with h5py.File(cropped_hdf5_path, 'w') as hf:
    hf.create_dataset("train_images", data=cropped_train_images, dtype=np.uint8)
    hf.create_dataset("train_labels", data=train_labels, dtype=np.uint8)
    hf.create_dataset("val_images", data=cropped_val_images, dtype=np.uint8)
    hf.create_dataset("val_labels", data=val_labels, dtype=np.uint8)
    hf.create_dataset("test_images", data=cropped_test_images, dtype=np.uint8)
    hf.create_dataset("test_labels", data=test_labels, dtype=np.uint8)


In [1]:
import h5py
import numpy as np

# Crop the centre 32x32 area of the image
def crop_center(img):
    y, x, _ = img.shape
    start_x = x//2 - 16
    start_y = y//2 - 16
    return img[start_y:start_y+32, start_x:start_x+32]

# Crop training, validation and test images
def crop_images(images):
    return np.array([crop_center(img) for img in images])

with h5py.File('split_data.h5', 'r') as hf:
    train_images = hf['train_images'][:]
    val_images = hf['val_images'][:]
    test_images = hf['test_images'][:]

cropped_train_images = crop_images(train_images)
cropped_val_images = crop_images(val_images)
cropped_test_images = crop_images(test_images)

output_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\cropped_data.h5'
with h5py.File(output_path, 'w') as hf:
    
    hf.create_dataset("train_images", data=cropped_train_images, dtype=np.uint8)
    hf.create_dataset("val_images", data=cropped_val_images, dtype=np.uint8)
    hf.create_dataset("test_images", data=cropped_test_images, dtype=np.uint8)

    hf.create_dataset("train_labels", data=hf['train_labels'][:], dtype=np.uint8)
    hf.create_dataset("val_labels", data=hf['val_labels'][:], dtype=np.uint8)
    hf.create_dataset("test_labels", data=hf['test_labels'][:], dtype=np.uint8)


KeyError: "Unable to open object (object 'train_labels' doesn't exist)"

In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
print(torch.cuda.is_available())

False


In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torchvision.models.resnet import ResNet18_Weights

# Define the model structure
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Load the model weights
model_weights_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\1st_train_model.pth'
model.load_state_dict(torch.load(model_weights_path))

# Print the model summary
model_structure = model_structure.to(device)
model_summary = summary(model_structure, input_size=(3, 224, 224))
model_summary


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [1]:
import torch
print(torch.backends.cudnn.enabled)

True


In [10]:
from PIL import Image
import os
import h5py
import numpy as np

def center_crop_and_save_to_h5(input_dir, output_path):
    images_list = []
    
    # 遍历指定文件夹中的所有图片
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.tif'):  # 只处理 .tif 文件
            file_path = os.path.join(input_dir, file_name)
            image = Image.open(file_path)
            
            # 计算裁剪的起始坐标
            left = (image.width - 32) / 2
            top = (image.height - 32) / 2
            right = (image.width + 32) / 2
            bottom = (image.height + 32) / 2
            
            # 裁剪图片
            cropped_image = image.crop((left, top, right, bottom))
            images_list.append(np.array(cropped_image))
    
    # 将图像列表转换为numpy数组
    images_array = np.array(images_list)
    
    # 保存为.h5文件
    with h5py.File(output_path, 'w') as hf:
        hf.create_dataset("images", data=images_array)

# 指定输入文件夹和输出.h5文件路径
input_directory = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\test'
output_h5_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\test_data.h5'
center_crop_and_save_to_h5(input_directory, output_h5_path)

In [19]:
import h5py
import numpy as np
from PIL import Image
from torchvision import transforms

# 归一化参数
normalized_mean = 165 / 255
normalized_std = 65 / 255
PIXEL_MEAN = [normalized_mean, normalized_mean, normalized_mean]
PIXEL_STD = [normalized_std, normalized_std, normalized_std]

# 定义归一化的transform
normalize_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PIXEL_MEAN, std=PIXEL_STD)
])

# 输入和输出的文件路径
input_path = 'C:\\Users\\dengz\\Desktop\\DeepLearninig7318\\A2\\histopathologic-cancer-detection\\test_data.h5'
output_path = 'C:\\Users\\dengz\\Desktop\\DeepLearninig7318\\A2\\histopathologic-cancer-detection\\test_data_normal.h5'

# 读取数据
with h5py.File(input_path, 'r') as f_in:
    images = f_in['images'][:]
    image_names = f_in['file_names'][:]
    
    # 对每张图像进行归一化
    normalized_images = []
    for img in images:
        pil_img = Image.fromarray(img)
        normalized_img = normalize_transform(pil_img)
        normalized_images.append(normalized_img.numpy())

    normalized_images = np.array(normalized_images)

    # 保存到新的HDF5文件中
    with h5py.File(output_path, 'w') as f_out:
        f_out.create_dataset('images', data=normalized_images)
        f_out.create_dataset('file_names', data=image_names)
